In [6]:
import shap
import spacetimeformer as stf
import sys

In [8]:
sys.path.append('../../bats_transformer')

In [9]:
from data.bats_dataset import *

In [3]:
model_path = "../../bats_transformer/models/daytime_files_new_10_11/models_31"

In [5]:
model = stf.spacetimeformer_model.Spacetimeformer_Forecaster(max_seq_len = 54).load_from_checkpoint(checkpoint_path=model_path)

Forecaster
	L2: 0.001
	Linear Window: 0
	Linear Shared Weights: False
	RevIN: False
	Decomposition: False
GlobalSelfAttn: AttentionLayer(
  (inner_attention): PerformerAttention(
    (kernel_fn): ReLU()
  )
  (query_projection): Linear(in_features=200, out_features=200, bias=True)
  (key_projection): Linear(in_features=200, out_features=200, bias=True)
  (value_projection): Linear(in_features=200, out_features=200, bias=True)
  (out_projection): Linear(in_features=200, out_features=200, bias=True)
  (dropout_qkv): Dropout(p=0.0, inplace=False)
)
GlobalCrossAttn: AttentionLayer(
  (inner_attention): PerformerAttention(
    (kernel_fn): ReLU()
  )
  (query_projection): Linear(in_features=200, out_features=200, bias=True)
  (key_projection): Linear(in_features=200, out_features=200, bias=True)
  (value_projection): Linear(in_features=200, out_features=200, bias=True)
  (out_projection): Linear(in_features=200, out_features=200, bias=True)
  (dropout_qkv): Dropout(p=0.0, inplace=False)
)
L

In [10]:
ignore_cols = ["FreqLedge","AmpK@end", "Fc", "FBak15dB  ", "FBak32dB", "EndF", "FBak20dB", "LowFreq", "Bndw20dB", 
               "CallsPerSec", "EndSlope", "SteepestSlope", "StartSlope", "Bndw15dB", "HiFtoUpprKnSlp", "HiFtoKnSlope", 
               "DominantSlope", "Bndw5dB", "PreFc500", "PreFc1000", "PreFc3000", "KneeToFcSlope", "TotalSlope", 
               "PreFc250", "CallDuration", "CummNmlzdSlp", "DurOf32dB", "SlopeAtFc", "LdgToFcSlp", "DurOf20dB", "DurOf15dB", 
               "TimeFromMaxToFc", "KnToFcDur", "HiFtoFcExpAmp", "AmpKurtosis", "LowestSlope", "KnToFcDmp", "HiFtoKnExpAmp", 
               "DurOf5dB", "KnToFcExpAmp", "RelPwr3rdTo1st", "LnExpB_StartAmp", "Filter", "HiFtoKnDmp", "LnExpB_EndAmp", 
               "HiFtoFcDmp", "AmpSkew", "LedgeDuration", "KneeToFcResidue", "PreFc3000Residue", "AmpGausR2", "PreFc1000Residue", 
               "Amp1stMean", "LdgToFcExp", "FcMinusEndF", "Amp4thMean", "HiFtoUpprKnExp", "HiFtoKnExp", "KnToFcExp", "UpprKnToKnExp", 
               "Kn-FcCurviness", "Amp2ndMean", "Quality", "HiFtoFcExp", "LnExpA_EndAmp", "RelPwr2ndTo1st", "LnExpA_StartAmp", 
               "HiFminusStartF", "Amp3rdMean", "PreFc500Residue", "Kn-FcCurvinessTrndSlp", "PreFc250Residue", "AmpVariance", "AmpMoment", 
               "meanKn-FcCurviness", "MinAccpQuality", "AmpEndLn60ExpC", "AmpStartLn60ExpC", "Preemphasis", "MaxSegLnght" ,"Max#CallsConsidered" ]
ignore_cols += ["Filename", "NextDirUp", 'Path', 'Version', 'Filter', 'Preemphasis', 'MaxSegLnght', "ParentDir", "file_id", "chirp_idx", "split"]

In [25]:
data_module = stf.data.DataModule(
    datasetCls = BatsCSVDatasetWithMetadata,
    dataset_kwargs = {
        "root_path": "../../bats_transformer/data/daytime_files_new/splits",
        "prefix": "split",
        "ignore_cols": ignore_cols,
        "metadata_cols": ["file_id", "chirp_idx"],
        "time_col_name": "TimeIndex",
        "val_split": 0.05,
        "test_split": 0.05,
        "context_points": None,
        "target_points": 1,
    },
    batch_size=64,
    workers=4
)

In [31]:
train_data = data_module.train_dataloader()

In [38]:
batch = next(iter(train_data))  # Get a batch of data
batch = batch[0]
batch.shape

torch.Size([64, 31, 1])

In [39]:
# Assuming you have a sample input batch 'X' for the model
# Replace 'X' with your actual input data (as a numpy array or torch tensor)

explainer = shap.DeepExplainer(model, batch)
shap_values = explainer.shap_values(batch)

# To visualize feature importance for the first sample
shap.summary_plot(shap_values, batch)

TypeError: forward() missing 3 required positional arguments: 'y_c', 'x_t', and 'y_t'